In [1]:
import numpy as np
import matplotlib.pyplot as plt
import visualization
import os
from gta_math import points_to_homo, ndc_to_view, construct_proj_matrix, view_to_world, construct_view_matrix
from visualization import load_depth, save_pointcloud_csv
import progressbar
from pointcloud_to_voxelmap import pointclouds_to_voxelmap
from joblib import Parallel, delayed
from configparser import ConfigParser
from PIL import Image
import pickle

In [2]:
ini_file = "gta-postprocessing.ini"
visualization.multi_page = False
visualization.ini_file = ini_file

conn = visualization.get_connection()
cur = conn.cursor()

CONFIG = ConfigParser()
CONFIG.read(ini_file)
in_directory = CONFIG["Images"]["Tiff"]
out_directory = CONFIG["Images"]["MlDatasetVoxel"]


In [3]:
run_id = 6

cur.execute("""SELECT DISTINCT scene_id \
  FROM snapshots \
  WHERE run_id = {} \
  """.format(run_id))

scenes = []
for row in cur:
    res = dict(row)
    scenes.append(res)

print('There are {} scenes'.format(len(scenes)))


There are 8438 scenes


### functions for each scene

In [6]:
class NoMainImageException(Exception):
    pass

# import time
MAX_DISTANCE = 20 # in meters, meaning I care only for point up to 20 meters

def get_base_name(name):
    return os.path.basename(os.path.splitext(name)[0])


def scene_to_pointcloud(cameras):
    pointclouds = []
    cam_positions = []
    
    for cam in cameras:
        pointcloud = camera_to_pointcloud(cam)
        pointclouds.append(pointcloud)
        cam_positions.append(cam['camera_pos'])
    return pointclouds, cam_positions


def get_main_image_name(cameras):
    for cam in cameras:
        # this is the main camera
        if np.array_equal(cam['camera_relative_rotation'], [0, 0, 0]):
            return cam['imagepath']
    raise NoMainImageException('no main image')

    
def load_scene_db_data(scene_id):
    conn = visualization.get_connection()
    cur = conn.cursor()
    cur.execute("""SELECT snapshot_id, imagepath, cam_near_clip, camera_fov, width, height, \
      ARRAY[st_x(camera_relative_rotation), st_y(camera_relative_rotation), st_z(camera_relative_rotation)] as camera_relative_rotation, \
      ARRAY[st_x(camera_pos), st_y(camera_pos), st_z(camera_pos)] as camera_pos, \
      ARRAY[st_x(camera_rot), st_y(camera_rot), st_z(camera_rot)] as camera_rot \
      FROM snapshots \
      WHERE scene_id = '{}'
      ORDER BY timestamp ASC \
    """.format(scene_id))

    cameras = []
    for row in cur:
        res = dict(row)
        res['camera_rot'] = np.array(res['camera_rot'])
        res['camera_pos'] = np.array(res['camera_pos'])
        res['camera_relative_rotation'] = np.array(res['camera_relative_rotation'])
        res['view_matrix'] = construct_view_matrix(res['camera_pos'], res['camera_rot'])
        res['proj_matrix'] = construct_proj_matrix(res['height'], res['width'], res['camera_fov'], res['cam_near_clip'])
        cameras.append(res)
    return cameras


def camera_to_pointcloud(cam):
    name = cam['imagepath']
    depth = load_depth(name)
    cam['cam_far_clip'] = MAX_DISTANCE
    vecs, _ = points_to_homo(cam, depth)
    assert(vecs.shape[0] == 4)
    vecs_p = ndc_to_view(vecs, cam['proj_matrix'])
    vecs_p_world = view_to_world(vecs_p, cam['view_matrix'])
    assert(vecs_p_world.shape[0] == 4)
    return vecs_p_world[0:3, :]


def scene_to_voxelmap(scene_id):
    # start = time.time()
    pointclouds, cam_positions = scene_to_pointcloud(scene_id)
    # end = time.time()
    # print('scene to pointclouds', end - start)
    # start = time.time()
    assert(pointclouds[0].shape[0] == 3)
    voxelmap = pointclouds_to_voxelmap(pointclouds, cam_positions)
    # end = time.time()
    # print('pointclouds to voxelmap', end - start)

    return voxelmap


def convert_tiff(in_directory, out_directory, out_name, name):
    out_format = 'jpg'
    outfile = os.path.join(out_directory, "{}.{}".format(out_name, out_format))
    if os.path.exists(outfile):
        return
    try:
        infile = os.path.join(in_directory, name)
        im = Image.open(infile)
        im = im.convert(mode="RGB")
        # print("Generating new format for {} to new file {}".format(name, out_name))
        im.save(outfile)
    except OSError:
        # print("Skipping invalid file {}".format(name))
        return


def convert_scene_to_img_and_voxelmap(in_directory, out_directory, scene_id):
    if 'pbar' in globals() and 'counter' in globals():
        global counter
        counter += 1
        pbar.update(counter)

    # start = time.time()
    cameras = load_scene_db_data(scene_id)
    try:
        image_name = get_main_image_name(cameras)
        # end = time.time()
        # print('loading data from db', end - start)
        # start = time.time()
        convert_tiff(in_directory, out_directory, image_name, image_name+'.tiff')
        # end = time.time()
        # print('converting tiff to jpg', end - start)
        # start = time.time()
        outfile = os.path.join(out_directory, "{}.rick".format(image_name))
        if os.path.exists(outfile):
            return
        voxelmap = scene_to_voxelmap(cameras)
        # end = time.time()
        # print('scene to voxelmap', end - start)
        # start = time.time()
        with open(outfile, 'wb+') as f:
            pickle.dump(voxelmap, f)
        # end = time.time()
        # print('saving voxelmap', end - start)
    except NoMainImageException:
        print('No main image for scene {}, skipping.'.format(scene_id))


### actually running the extraction

In [ ]:
workers = 20

widgets = [progressbar.Percentage(), ' ', progressbar.Counter(), ' ', progressbar.Bar(), ' ',
           progressbar.FileTransferSpeed()]

pbar = progressbar.ProgressBar(widgets=widgets, maxval=len(scenes)).start()
counter = 0

Parallel(n_jobs=workers, backend='threading')(delayed(convert_scene_to_img_and_voxelmap)(in_directory, out_directory, i['scene_id']) for i in scenes)


  3% 261 |#                                                         | 165.5 B/s

No main image for scene 388309dc-568f-43ab-b2c1-86fb809cc570, skipping.


  3% 328 |##                                                        |   0.8 B/sC:\ProgramData\Anaconda3\lib\site-packages\tifffile\tifffile.py:2642: RuntimeWarning: py_decodelzw encountered unexpected end of stream
  strip = decompress(strip)
  4% 412 |##                                                        |   0.4 B/s

No main image for scene 592377fd-866a-4d81-bdb0-4e63ae584bab, skipping.


  5% 461 |###                                                       |   0.4 B/s

No main image for scene c70fe8d1-e628-483e-a27a-6a2c7b862857, skipping.


  6% 565 |###                                                       |   0.3 B/s

### running extraction for one scene

In [ ]:
convert_scene_to_img_and_voxelmap(in_directory, out_directory, scenes[0]['scene_id'])

In [ ]:
# this is to visualize voxelmap as a pointcloud
def voxelmap_to_pointcloud_csv(path_in, path_out):
    with open(path_in, 'rb') as f:
        voxels, values, voxel_size = pickle.load(f)
    voxels = voxels[:, values >= 0]
    save_pointcloud_csv(voxels.T, path_out)
    

def scene_to_pointcloud_csv(scene_id, path_out):
    if 'pbar' in globals() and 'counter' in globals():
        global counter
        counter += 1
        pbar.update(counter)

    cameras = load_scene_db_data(scene_id)
    pointclouds, cam_positions = scene_to_pointcloud(cameras)
    big_pointcloud = np.zeros((3, 0))
    for pc in pointclouds:
        big_pointcloud = np.append(big_pointcloud, pc, axis=1)
    save_pointcloud_csv(big_pointcloud.T, path_out)


In [ ]:
voxelmap_to_pointcloud_csv(r'D:\ml-datasets-voxel\2018-03-07--15-19-54--505.rick', r'D:\ml-datasets-voxel\30m-2018-03-07--15-19-54--505.csv')
scene_to_pointcloud_csv(scenes[0]['scene_id'], r'D:\ml-datasets-voxel\pc-2018-03-07--15-19-54--505.csv')